In [1]:
from multiprocessing import Pool
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def arrangement_1(nleds,d):
    l=math.sqrt(nleds);
    ledcord=np.zeros((2,nleds));
    temp=0;
    for k in range(nleds):
        ledcord[0,k]=-(l-1)*d/2 +((k)%l)*d;
        if (k)%l ==0 and k != 0:
            temp=temp+1;
        ledcord[1,k]=-(l-1)*d/2 + temp*d;
    return ledcord
    
def E_r(ledcord,nleds,Ir_0,z,d,r):    
    x=r;
    y=0;
    E=0;
    for i in range(nleds):
        den=np.square(np.square(x-ledcord[0,i]) +np.square(y-ledcord[1,i]) +z**2);
        E= E + (z**2 * Ir_0)*(np.reciprocal(den))
    return E
  

In [148]:
Ir_0 = 0.1;
final_list = []
def ratio(d):
    final_list = []
    for nleds in [9,25,49,81,121]:
        lled=np.sqrt(nleds)
        if((lled-1)*d>0.3):
            break
        else:     
            ledcord = arrangement_1(nleds,d)
            nz = (0.5 - (lled-1)*d)/0.001 +1
            for z in np.linspace((lled-1)*d,0.5,nz):
                E_cent = E_r(ledcord,nleds,Ir_0,z,d,0)
                E_edge = E_r(ledcord,nleds,Ir_0,z,d,0.04) 
                t = 4000/(E_cent+E_edge)
                percent_uniformity=(E_edge/E_cent)*100
                energy_edge = E_edge * t
                energy_cent = E_cent * t
                energy_uniformity = (energy_edge/energy_cent)*100
                final_list.append([nleds,d,z,E_cent,E_edge,t,energy_edge/10,energy_cent/10,percent_uniformity,])
    return final_list

with Pool(7) as p:
      l = list(tqdm(p.imap(ratio,np.linspace(0.01,0.2,191)), total=191))

100%|██████████| 191/191 [00:15<00:00, 12.12it/s]


In [149]:
t=[]
for i in l:
    t.extend(i)
#     print(len(i))
final_list=t

In [150]:
df = pd.DataFrame(final_list,columns=["nleds","d(m)","z(m)","E_cent(W/m^2)","E_edge(W/m^2)","exposure_time(s)","energy_edge(mJ/cm^2)","energy_cent(mJ^cm^2)","percent_uniformity"])

In [151]:
#pd.set_option("display.max_rows", None, "display.max_columns", None)
# df

In [152]:
idx = df.groupby(['nleds'])['percent_uniformity'].transform(max) == df['percent_uniformity']
df[idx].sort_values("nleds")

,nleds,d(m),z(m),E_cent(W/m^2),E_edge(W/m^2),exposure_time(s),energy_edge(mJ/cm^2),energy_cent(mJ^cm^2),percent_uniformity
97691,9,0.149,0.5,2.903416,2.879550,691.686591,199.174597,200.825403,99.177990
77293,25,0.075,0.5,8.476842,8.399254,237.021645,199.080491,200.919509,99.084699
60996,49,0.049,0.5,16.998545,16.835624,118.223682,199.036947,200.963053,99.041562
49243,81,0.037,0.5,28.289047,28.014223,71.043831,199.023776,200.976224,99.028518
39980,121,0.030,0.5,42.335684,41.922868,47.472926,199.020121,200.979879,99.024899


In [146]:
#ef=df[df["nleds"]==121]
#ef[ef['percent_uniformity']>99].sort_values("d(m)")

In [153]:
ef=df[df['percent_uniformity']>99]
idx = ef.groupby(['nleds'])['E_cent(W/m^2)'].transform(max) == ef['E_cent(W/m^2)']
ef[idx].sort_values("nleds")

,nleds,d(m),z(m),E_cent(W/m^2),E_edge(W/m^2),exposure_time(s),energy_edge(mJ/cm^2),energy_cent(mJ^cm^2),percent_uniformity
97607,9,0.149,0.415582,3.863076,3.824506,520.319643,198.996582,201.003418,99.001591
77262,25,0.075,0.469000,9.438445,9.344227,212.962242,198.996753,201.003247,99.001761
60982,49,0.049,0.485932,17.858319,17.679893,112.554903,198.995860,201.004140,99.000876
49234,81,0.037,0.491000,29.199150,28.907725,68.838670,198.996937,201.003063,99.001942
39972,121,0.030,0.492000,43.545605,43.110638,46.159398,198.996109,201.003891,99.001123
